In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
train.head()

In [ ]:
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
test.head()

First, we create counting measures for each word, that is to say the number of times they appear in all textes. The idea is that a rare word will likely be complex.

In [ ]:
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
# lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
stop_words = set(stopwords.words('english')) 
def lem(sentence):
    words = word_tokenize(sentence.lower()) 
    n = len(words)
    words = [ps.stem(w) for w in words if not w in stop_words] # lemmatizer.lemmatize
    return ' '.join(words), n-len(words)
lem('THE CATS are happier today than yesterday')

In [ ]:
train['stemmed'], train['count_stop_words'] = zip(*train['excerpt'].map(lem))
test['stemmed'], test['count_stop_words'] = zip(*test['excerpt'].map(lem))

In [ ]:
all_words = [w for c in (train['stemmed'], test['stemmed']) for t in c for w in t.split(' ')]
dic = {}
try:
    while True:
        w = all_words.pop()
        if not w in dic.keys():
            dic[w] = 1
        else:
            dic[w] +=1
except:
    pass

In [ ]:
max(dic.values()), min(dic.values())

In [ ]:
dic['stopword'] = max(dic.values()) + 1

In [ ]:
def f(row):
    s = [dic[w] for w in row['stemmed'].split(' ')]
#     st = row['count_stop_words']
    n = len(s) # + st
    return sum(s) / n, row['count_stop_words'] / n

In [ ]:
train['score'], train['count_stop_words'] = zip(*train.apply(f, axis=1))
train.head()

In [ ]:
test['score'], test['count_stop_words'] = zip(*test.apply(f, axis=1))
test.head()

In [ ]:
from sklearn.linear_model import LinearRegression
l = LinearRegression()
l.fit(train[['score', 'count_stop_words']].values.reshape(-1, 2), train['target'].values)
l.intercept_, l.coef_

In [ ]:
np.mean((l.predict(train[['score', 'count_stop_words']].values.reshape(-1, 2)) - train.target)**2)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(train.target.values)
plt.show()

In [ ]:
sub = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
sub['target'] = l.predict(test[['score', 'count_stop_words']].values.reshape(-1, 2))
sub.to_csv('submission.csv', index=False)